In [1]:
import polars as pl
from dotenv import load_dotenv
import os
from datetime import datetime
load_dotenv()

True

In [2]:
# config to load OS
os_files_path = os.getenv('OS_FILES_PATH')
csv_file_names = sorted(os.listdir(os_files_path))
os.makedirs('output', exist_ok=True)
record_types = [21,23,24,28,31,32]

In [4]:
# get commercial props uprn list
df_32 = pl.read_csv(f'{os_files_path}/ID32_Class_Records.csv', infer_schema_length=0)
df_32_commercial = df_32.filter(
    pl.col('CLASSIFICATION_CODE').str.starts_with('C') | 
    pl.col('CLASS_SCHEME').str.starts_with('VOA')
)

In [5]:
# read each file and assign to correct os record type
for file_name in csv_file_names:
    record_type = int(file_name[2:4])
    if record_type not in record_types: continue
    print(f'file_name: [{file_name}], time: [{datetime.now().strftime("%H:%M:%S")}]')
    temp_df = pl.read_csv(f'{os_files_path}/{file_name}', infer_schema_length=0)
    # reduce size a bit
    if record_type == 23:
        temp_df = temp_df.filter(pl.col('SOURCE') == '7666VN')
    if record_type == 32:
        temp_df = df_32_commercial
    else:
        temp_df = temp_df.join(df_32_commercial, on='UPRN', how='semi')
    
    # save to parquet file
    temp_df.write_parquet(f'output/os_recordtype_{record_type}.parquet')

print('done')

file_name: [ID21_BLPU_Records.csv], time: [12:11:47]
file_name: [ID23_XREF_Records.csv], time: [12:12:02]
file_name: [ID24_LPI_Records.csv], time: [12:12:48]
file_name: [ID28_DPA_Records.csv], time: [12:12:58]
file_name: [ID31_Org_Records.csv], time: [12:13:02]
file_name: [ID32_Class_Records.csv], time: [12:13:03]
done
